In [1]:
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
from sklearn import metrics, linear_model
from random import choices
from itertools import compress
from sklearn.preprocessing import StandardScaler, Normalizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy import stats

class Data(object): 
    def __init__(self):
        pass
    
Data.X = Data()
Data.X.train = pd.read_csv('./Data/Sets/train_GSE36961.csv', index_col=0)
Data.X.test = pd.read_csv('./Data/Sets/test_rna_seq_data.csv', index_col=0)
Data.y = Data()
Data.y.train = list(pd.read_csv('./Data/Sets/train_GSE36961_target.csv').iloc[:,1])
Data.y.test = list(pd.read_csv('./Data/Sets/test_rna_seq_target.csv').iloc[:,1])

class FeatureExtraction(object):
    """
    Класс для экстракции фичей. Главная идея не фиксировать случайность, а оседлать её :)

    """

    def __init__(self):
        pass

    def fit_inner_loop(self, X_train, X_test, y_train, y_test, C=0.03):

        """
        Будем n_iter раз бутстрепить сбалансированную train выборку из X_train.
        Обучаем лог.рег. с L1-решуляризацией, с коэффициентом как мы отобрали выше.
        Тестим на X_test, значение добавляем в roc_auc_list

        Если на X_test модель работает круче 0.7, то: 
            1) ненулевые фичи модели добавляем в словарик отобранных фичей feature_dict
            2) обновляем число фичей в перменной len_best_feature = len(feature_dict.keys())
        Если нет, то:
            3) дублируем последнее значение в len_best_feature, т.к. число фичей не изменилось
        """

        len_best_feature = [0]  # заводим лист, в котором будем отслеживать изменение количества фичей
        len_best_more_one = [
            0]  # заводим лист, в котором будем отслеживать изменение количества числа включений уже включенных фичей
        roc_auc_list = list()  # аналогично, отслеживаем как меняется roc-auc, так для интереса
        feature_dict = dict()  # # словарь "ген: log.reg.coef"

        # чтобы получить сбаланнсированную выборку, бутстрепим отдельно семплы из контроля и из опыта

        mask = np.array(y_train) == 0

        k_len = min(len(mask) - sum(mask), sum(mask))  # размер выборки бутстрепа, берем размер минимальной группы HCM или CTRL

        CTRL_rows = list(compress(range(0, len(mask)), mask))
        HCM_rows = list(compress(range(0, len(mask)), mask == False))

        _HCM_rows = choices(HCM_rows, k=k_len)  # бутстрепим номера строк из группы больных
        _CTRL_rows = choices(CTRL_rows, k=k_len)  # бутстрепим номера строк из группы здоровых

        # объединяем это всё дело обратно

        _X_train = pd.DataFrame(X_train).iloc[_HCM_rows + _CTRL_rows, :]
        _y_train = np.array(y_train)[_HCM_rows + _CTRL_rows]
        print(_X_train.index)

        # обучаем лог.рег. с ранее отобранным коэффициентом регуляризации

        linear_regressor = linear_model.LogisticRegression(penalty='l1', C=C, solver='liblinear',
                                                           random_state=42)
        linear_regressor.fit(_X_train, _y_train)

        # тестим

        roc_auc = metrics.roc_auc_score(y_score=linear_regressor.predict(X_test), y_true=y_test)
        roc_auc_list.append(roc_auc)

        # далее отбираем фичи из моделей, которые хоть как-то работают (roc_auc > 0.7)

        if roc_auc > 0.7:
            # отбираем смысловые фичи
            mask = linear_regressor.coef_ != 0
            genes = X_train.columns[mask[0]]
            values = linear_regressor.coef_[mask]

            _feature_dict = dict(zip(genes, abs(values) * roc_auc))  # делаем временный словарь "ген: его ценность"

            # обнавляем глобальный словарь фичей
            for gene, values in _feature_dict.items():
                if gene in feature_dict:
                    feature_dict[gene].append(values)
                else:
                    feature_dict[gene] = [values]

            len_best_feature.append(len(feature_dict.keys()))

            feature_distr = np.array(list(map(lambda x: len(feature_dict[x]), feature_dict)))
            len_best_more_one.append(sum(feature_distr[feature_distr > 1]))

        else:
            # если модель была говёной, то просто дублируем предыдущее значение. Ну, число фичей то не изменилось :)
            len_best_feature.append(len_best_feature[-1])
            len_best_more_one.append(len_best_more_one[-1])

        self.len_best_more_one = np.array(len_best_more_one)
        self.len_best_feature = np.array(len_best_feature)
        self.feature_dict = feature_dict
        self.roc_auc_list = roc_auc_list

    def fit_all_loops(self, X, y, inner_itter, extr_itter):

        """
        Идея: повторим экстракцию фичей n раз,
        отсортируем каждый из получившихся наборов по важности фичей, найдем размер окна для отбора n-топ фичей,
        в котором состав фичей минимально изменяется от набора к набору. А потом отберем те фичи, которые всегда встречаются в окне этого размера.
        """

        list_feature_dicts = list()

        for i in range(0, extr_itter):
            print(i, 'external loop fitting...', sep=' ')
            # train_test split and transformation
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y,
                                                                test_size=0.2,
                                                                random_state=i)

            normalizer = Normalizer()

            X_train = pd.DataFrame(normalizer.fit_transform(X_train), columns=X_train.columns)
            X_test = pd.DataFrame(normalizer.fit_transform(X_test), columns=X_test.columns)

            scaler = StandardScaler()
            X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
            X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

            # отбор фичей
            fe = FeatureExtraction()
            fe.fit_inner_loop(inner_itter,
                              X_train, X_test, y_train, y_test)
            list_feature_dicts.append(fe.feature_dict)

        self.list_feature_dicts = list_feature_dicts


In [19]:
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
from sklearn import metrics, linear_model
from random import choices
from itertools import compress
from sklearn.preprocessing import StandardScaler, Normalizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy import stats
from multiprocessing import Manager, Pool, cpu_count

NUM_ITERATIONS = 200
NUM_PER_FILE = 50
NUM_PROCESSES = 4

class Data(object): 
    def __init__(self):
        pass
    
Data.X = Data()
Data.X.train = pd.read_csv('./Data/Sets/train_GSE36961.csv', index_col=0)
Data.y = Data()
Data.y.train = list(pd.read_csv('./Data/Sets/train_GSE36961_target.csv').iloc[:,1])

X_shared = pd.DataFrame(Data.X.train.transpose())
y_shared = Data.y.train

def MC_model(X, y, C=0.03):
#        X = X_shared, 
#        y = y_shared

        mask = np.array(y) == 0

        # размер выборки бутстрепа, берем размер минимальной группы HCM или CTRL
        k_len = min(len(mask) - sum(mask), sum(mask))  

        CTRL_idx = list(compress(range(0, len(mask)), mask))
        HCM_idx = list(compress(range(0, len(mask)), mask == False))

        train_idx = choices(HCM_idx, k=k_len) + choices(CTRL_idx, k=k_len) # бутстрепим номера строк из группы больных и здоровых
        test_idx = list(set([i for i in range(len(mask))]) - set(train_idx))

        # объединяем это всё дело обратно

        _X_train = pd.DataFrame(X).iloc[train_idx, :]
        _y_train = np.array(y)[train_idx]
        _X_test = pd.DataFrame(X).iloc[test_idx, :]
        _y_test = np.array(y)[test_idx]
        
        scaler = StandardScaler()
        _X_train = scaler.fit_transform(_X_train)
        _X_test = scaler.transform(_X_test)

        # обучаем лог.рег. с ранее отобранным коэффициентом регуляризации

        linear_regressor = linear_model.LogisticRegression(penalty='l1', C=C, solver='liblinear',
                                                           random_state=42)
        linear_regressor.fit(_X_train, _y_train)

        # тестим

        roc_auc = metrics.roc_auc_score(y_score=linear_regressor.predict(_X_test), y_true=_y_test)

        # далее отбираем фичи из моделей, которые хоть как-то работают (roc_auc > 0.7)

        if roc_auc > 0.7:
            # отбираем смысловые фичи
            mask = linear_regressor.coef_ != 0
            mask = np.append(mask[0], roc_auc)
        
        return mask

def run_iteration(config):
#     val = MC_model(X_shared, y_shared)
    val = MC_model(config[0], config[1])
    return val

# mgr = Manager()
# ns = mgr.Namespace()
# ns.X = X_shared
# ns.y = y_shared

np.random.seed(43)

for i in range(NUM_ITERATIONS // NUM_PER_FILE):
    print("Iteration:", i, "/", NUM_ITERATIONS // NUM_PER_FILE)
    
    with Pool(NUM_PROCESSES) as p:
        res = p.map(run_iteration, [(X_shared, y_shared) for _ in range(i*NUM_PER_FILE, (i+1)*NUM_PER_FILE)])
        
    out_df = pd.DataFrame.from_records(res)
    if i == 0:
        # create the initial file
        # write the data in a form of pandas data frame
        out_df.to_csv('./MC_res2.csv', header=False, index = False)
    else:
        # append it to the file
        out_df.to_csv('./MC_res2.csv', mode='a', header=False, index = False)

Iteration: 0 / 4
Iteration: 1 / 4
Iteration: 2 / 4
Iteration: 3 / 4


In [ ]:
import datetime
print(datetime.datetime.now())

In [54]:
mc_res = pd.read_csv('./MC_res2.csv', header=None)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [105]:
mc_res.loc[:,mc_res.dtypes == 'O'][0].astype('float')

ValueError: could not convert string to float: 'False'

In [90]:
len(mc_res.mean(axis=0))

37702

In [65]:
data = pd.DataFrame(np.matrix(mc_res)).fillna(0)

In [92]:
pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   ['np.nan', 'np.nan', np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                  columns=list("ABCD")).mean()

C    NaN
D    2.5
dtype: float64

In [47]:
mc_res = mc_res.fillna(0)

In [48]:
len(mc_res.mean(axis=0))

37702

In [23]:
Data.X.train.transpose().columns[mask]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 37846 but corresponding boolean dimension is 37701

In [9]:
mc_res1 = pd.read_csv('./MC_res.csv')

In [10]:
len(mc_res.columns)

37847

In [13]:
mc_data = mc_res.append(mc_res1, sort = False)

In [14]:
len(mc_data)

19998

In [15]:
len(mc_data.columns)

37849

In [34]:
mask = (np.array(mc_res1.mean(axis=0)[:-1]) + np.array(mc_res.mean(axis=0)[:-1]) > 0)

In [36]:
Data.X.train.transpose().columns[mask]

Index(['ACE2', 'APOA1', 'ATP1A2', 'C21ORF7', 'CA3', 'CENPA', 'CLIC6', 'EIF1AY',
       'FRZB', 'HS.576694', 'HSPA2', 'IER3', 'LOC100008589', 'MLLT11', 'MXRA5',
       'NPPA', 'NPPB', 'PROS1', 'RASD1', 'RASL11B', 'S100A9', 'SERPINA3',
       'SFRP1', 'SMOC2', 'THBS4', 'TPM3'],
      dtype='object', name='ID_REF')

In [4]:
# from multiprocessing import Manager, Pool, cpu_count

# def run_iteration(seed):
#     np.random.seed(42)
#     val = MC_model(ns.X, ns.y)
#     return val

# mgr = Manager()
# ns = mgr.Namespace()
# ns.X = X_shared
# ns.y = y_shared

# with Pool(8) as p:
#     res = p.map(run_iteration, [seed for seed in range(0, 200000)])

# res[0]